In [1]:
install.packages("leaps")
install.packages("Metrics")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library(tidyverse)
library(lubridate)

library(ggplot2)
library(broom)
library(tidyr)
library(dplyr)
library(Metrics)
library(modelr)

library(leaps)
library(caret)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘modelr’


The following objects are masked from ‘package:Metrics’:

    mae, mape, mse, rmse


The following object is masked from ‘package:broom’:

    bootstrap


Loading required package: lattice


Attaching package: ‘caret’


The following objects are masked from ‘package:Metrics’:

    precision, recall


The following object is masked from ‘package:purrr’:

    lift




- In the previous assignment, I explored the data with useful visuals that allowed me to observe relationships between input variables and response one and among input variables themselves. I also observed how some input variables were highly correlated (multicollinearity) by plotting a heatmap.
- in this assignment, I will first convert the 'Created_At' and 'Updated_AT' Date variables to numerical values, where they represent the days since created, updated respectively.
- Then, I will clean the Data and use the Forward Selection Algorithm to find the best-fit model.
- Afterward, I am going to split the data into training, testing data with a ratio of 80/20.
- then, I am going to use the variables from the Forward Selection results to fit an Additive linear regression model.
- Then, using the predict function, I am going to predict the number of stars in the testing set using the fitted model.
- Finally, I will calculate the RMSE to evaluate the accuracy of the prediction model.*


In [3]:
df <- read_csv('repositories.csv')

Rows: 215029 Columns: 24
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): Name, Description, URL, Homepage, Language, License, Topics, Defau...
dbl  (5): Size, Stars, Forks, Issues, Watchers
lgl  (9): Has Issues, Has Projects, Has Downloads, Has Wiki, Has Pages, Has ...
dttm (2): Created At, Updated At

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
df_subset <- df %>% sample_n(size = 10000)

head(df_subset, n = 3)

Name,Description,URL,Created At,Updated At,Homepage,Size,Stars,Forks,Issues,⋯,Has Issues,Has Projects,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch
<chr>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
XTTable,A tool for quickly laying out and configuring UITable and UICollectionView,https://github.com/Akateason/XTTable,2015-12-07 10:30:31,2023-05-31 09:29:10,NA,2949,208,40,0,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master
OBD2-Scantool,A scantool to read OBD2 info from cars.,https://github.com/AustinMurphy/OBD2-Scantool,2011-11-27 22:28:43,2023-09-20 15:25:07,NA,99,191,53,2,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master
LLMStack,"No-code platform to build generative AI apps, chatbots and agents with your data",https://github.com/trypromptly/LLMStack,2023-08-06 00:39:25,2023-09-25 14:04:37,https://llmstack.ai,48701,436,28,4,⋯,TRUE,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,main


* - I am choosing only those variables because of the limitation on running the Forward Selection Algorithm on all of the given variables. Also, the other columns (e.g. logical variables) contain very redundant data.  
Note: I explain more about this issue in the cell just above the Forward Selection cell down below.*

In [24]:
features <- c("Size", "Stars", "Forks",
              "Issues", 'Watchers', 'Is Archived', 'Has Issues', 'Has Pages',
              'Created At', 'Updated At')
df_subset_1 <- df_subset %>% select(all_of(features))

head(df_subset_1, n = 3)
nrow(df_subset_1)
sum(is.na(df_subset_1))

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,Created At,Updated At
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<date>,<date>
2949,208,40,0,208,FALSE,TRUE,FALSE,2015-12-07,2023-05-31
99,191,53,2,191,FALSE,TRUE,FALSE,2011-11-27,2023-09-20
48701,436,28,4,436,FALSE,TRUE,FALSE,2023-08-06,2023-09-25


[1] 10000

[1] 0

In [6]:
df_subset$'Created At' <- as.Date(strptime(df_subset$'Created At', format = "%Y-%m-%d %H:%M:%S"))
df_subset$'Updated At' <- as.Date(strptime(df_subset$'Updated At', format = "%Y-%m-%d %H:%M:%S"))

In [7]:
df_subset_2 <- df_subset_1 %>%
  mutate(
    `Created At` = as.Date(`Created At`),        
    `Updated At` = as.Date(`Updated At`),        
    repo_age_days = as.numeric(Sys.Date() - `Created At`),
    days_since_update = as.numeric(Sys.Date() - `Updated At`)
  )

head(df_subset_2, n = 3)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,Created At,Updated At,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<date>,<date>,<dbl>,<dbl>
2949,208,40,0,208,FALSE,TRUE,FALSE,2015-12-07,2023-05-31,3280,548
99,191,53,2,191,FALSE,TRUE,FALSE,2011-11-27,2023-09-20,4751,436
48701,436,28,4,436,FALSE,TRUE,FALSE,2023-08-06,2023-09-25,481,431


In [8]:
df_subset_3 <- df_subset_2 %>% select(-c('Created At', 'Updated At'))

head(df_subset_3, n = 3)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
2949,208,40,0,208,FALSE,TRUE,FALSE,3280,548
99,191,53,2,191,FALSE,TRUE,FALSE,4751,436
48701,436,28,4,436,FALSE,TRUE,FALSE,481,431


In [9]:
non_na_repo_age <- df_subset_3 %>% filter(!is.na(repo_age_days))
non_na_days_since_update <- df_subset_3 %>% filter(!is.na(days_since_update))

head(df_subset_3 %>% select(repo_age_days, days_since_update, n = 3))

repo_age_days,days_since_update,n
<dbl>,<dbl>,<dbl>
3280,548,40
4751,436,53
481,431,28
4002,590,30
3198,432,56
3643,433,165


In [10]:
df_cleaned <- df_subset_3 %>% filter(!is.na(Stars) & !is.na(Forks) & !is.na(repo_age_days))  

head(df_cleaned, n = 3)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
2949,208,40,0,208,FALSE,TRUE,FALSE,3280,548
99,191,53,2,191,FALSE,TRUE,FALSE,4751,436
48701,436,28,4,436,FALSE,TRUE,FALSE,481,431


In [11]:
summary_stats <- df_cleaned %>%
  summarize(
    count = n(),
    avg_stars = mean(Stars, na.rm = TRUE),
    avg_forks = mean(Forks, na.rm = TRUE),
    avg_issues = mean(Issues, na.rm = TRUE),
    avg_repo_age = mean(repo_age_days, na.rm = TRUE),
    avg_days_since_update = mean(days_since_update, na.rm = TRUE)
  )

print(summary_stats)
head(df_cleaned, n = 3)
nrow(df_cleaned)
sum(is.na(df_cleaned))

# A tibble: 1 × 6
  count avg_stars avg_forks avg_issues avg_repo_age avg_days_since_update
  <int>     <dbl>     <dbl>      <dbl>        <dbl>                 <dbl>
1 10000     1007.      202.       36.0        2744.                  453.


Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
2949,208,40,0,208,FALSE,TRUE,FALSE,3280,548
99,191,53,2,191,FALSE,TRUE,FALSE,4751,436
48701,436,28,4,436,FALSE,TRUE,FALSE,481,431


[1] 10000

[1] 0

In this cell, I will split the data into training/testing data (80/20 ratio). I will use them to fit the linear regression model, predict 'Stars' values, and evaluate the model.  

In [12]:
set.seed(123) # DO NOT CHANGE!

GitHub_sample <-
    df_cleaned %>%
    mutate(id = row_number())

training_GitHub <- 
    GitHub_sample %>%
    slice_sample(prop = 0.80, replace = FALSE)

testing_GitHub <- 
    GitHub_sample %>% 
    anti_join(training_GitHub, by = "id") %>%
    select(-id)

training_GitHub <- 
    training_GitHub %>% 
    select(-id)

head(training_GitHub)
nrow(training_GitHub)

nrow(testing_GitHub)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
4131,1513,102,51,1513,FALSE,TRUE,FALSE,2703,433
1202,538,76,45,538,FALSE,TRUE,FALSE,3290,431
380,219,24,6,219,FALSE,TRUE,FALSE,3055,547
8082,931,671,7,931,FALSE,TRUE,FALSE,2475,432
19,538,43,3,538,FALSE,TRUE,FALSE,1370,434
443338,260,148,1,260,FALSE,TRUE,FALSE,1921,433


[1] 8000

[1] 2000

- One of the limitations I faced when running the Forward Selection Algorithm on the given data was that The kernel would collapse and restart entirely.
- To solve this problem, I will drop some of the categorical variables that have many levels (>2) and I, from the experience I gained in this course, believe they have little to no utility in predicting the number of stars.

In [13]:
GitHub_forward_sel <- regsubsets(
  Stars ~ ., data = training_GitHub,
  method = "forward",
  nvmax = ncol(training_GitHub) - 1
)
GitHub_forward_sel

GitHub_fwd_summary <- summary(GitHub_forward_sel)

GitHub_fwd_summary <- tibble(
  n_input_variables = 1:length(GitHub_fwd_summary$rss),  
  RSS = GitHub_fwd_summary$rss,
  BIC = GitHub_fwd_summary$bic,
  Cp = GitHub_fwd_summary$cp
)

Subset selection object
Call: regsubsets.formula(Stars ~ ., data = training_GitHub, method = "forward", 
    nvmax = ncol(training_GitHub) - 1)
9 Variables  (and intercept)
                  Forced in Forced out
Size                  FALSE      FALSE
Forks                 FALSE      FALSE
Issues                FALSE      FALSE
Watchers              FALSE      FALSE
`Is Archived`TRUE     FALSE      FALSE
`Has Issues`TRUE      FALSE      FALSE
`Has Pages`TRUE       FALSE      FALSE
repo_age_days         FALSE      FALSE
days_since_update     FALSE      FALSE
1 subsets of each size up to 9
Selection Algorithm: forward

In [14]:
GitHub_fwd_summary
summary(GitHub_forward_sel)

n_input_variables,RSS,BIC,Cp
<int>,<dbl>,<dbl>,<dbl>
1,1.522173e-19,-546543.4,16.5675151
2,1.518752e-19,-546552.5,0.5579650
3,1.518428e-19,-546545.2,0.8531394
4,1.518212e-19,-546537.3,1.7156305
5,1.518023e-19,-546529.3,2.7214185
6,1.517895e-19,-546521.0,4.0451975
7,1.517889e-19,-546512.1,6.0138368
8,1.517887e-19,-546503.1,8.0055957
9,1.517886e-19,-546494.1,10.0000000


Subset selection object
Call: regsubsets.formula(Stars ~ ., data = training_GitHub, method = "forward", 
    nvmax = ncol(training_GitHub) - 1)
9 Variables  (and intercept)
                  Forced in Forced out
Size                  FALSE      FALSE
Forks                 FALSE      FALSE
Issues                FALSE      FALSE
Watchers              FALSE      FALSE
`Is Archived`TRUE     FALSE      FALSE
`Has Issues`TRUE      FALSE      FALSE
`Has Pages`TRUE       FALSE      FALSE
repo_age_days         FALSE      FALSE
days_since_update     FALSE      FALSE
1 subsets of each size up to 9
Selection Algorithm: forward
         Size Forks Issues Watchers `Is Archived`TRUE `Has Issues`TRUE
1  ( 1 ) " "  " "   " "    "*"      " "               " "             
2  ( 1 ) " "  "*"   " "    "*"      " "               " "             
3  ( 1 ) " "  "*"   " "    "*"      " "               " "             
4  ( 1 ) " "  "*"   " "    "*"      " "               " "             
5  ( 1 ) " "  "*"   " 

*based on the forward selection, the variables for the best-fit model are Size Forks Issues Watchers. Therefore I will fit the training data into a linear regression prediction model using an additive model for all the variables we just selected(4). the reason I selected model (4) is that it has the best combination of lowest CP, BIC and the among the highest RSS.*

In [15]:
GitHub_full <- lm(Stars ~ Size + Forks + Issues + Watchers, data = training_GitHub)

GitHub_full


Call:
lm(formula = Stars ~ Size + Forks + Issues + Watchers, data = training_GitHub)

Coefficients:
(Intercept)         Size        Forks       Issues     Watchers  
          0            0            0            0            1  


In [16]:
GitHub_test_pred_full <- predict(GitHub_full, newdata = testing_GitHub)

head(GitHub_test_pred_full)

1    2    3    4    5    6 
 418 2838  532 1029  289  206

In [17]:
GitHub_RMSE_model <- tibble(
    Model = "Full Regression",
    RMSE = rmse(model = GitHub_full,
                data = testing_GitHub))

GitHub_RMSE_model 

Model,RMSE
<chr>,<dbl>
Full Regression,0


* - By using the vif function, and to resolve multicollinearity, I got the following results:
  - Size: 1.001, Forks: 1.615, Issues: 1.441, Watchers: 2.042, `Has Discussions`: 1.051repo_age_days: 1.087, days_since_update: 1.071.
  - I saw that 'Watchers' has a strong correlation with both 'Forks' and 'Issues' (0.62 and 0.55 respectively).
  - Now lets fit a linear regression model without 'Watchers'
  - Note: I could have removed 'Watchers' before running the Forward Selection Algorithm, however, I wanted to see what will the algorithm select as the best-fit model (and it included 'Watchers')*

In [18]:
GitHub_No_Watchers <- lm(Stars ~ Size + Forks + Issues + `Is Archived`, data = training_GitHub)

GitHub_No_Watchers


Call:
lm(formula = Stars ~ Size + Forks + Issues + `Is Archived`, data = training_GitHub)

Coefficients:
      (Intercept)               Size              Forks             Issues  
        3.737e+02         -7.477e-05          3.318e+00         -1.086e+00  
`Is Archived`TRUE  
       -1.976e+02  


In [19]:
GitHub_pred_No_Watchers <- predict(GitHub_No_Watchers, newdata = testing_GitHub)

head(GitHub_pred_No_Watchers)

1         2         3         4         5         6 
 815.0206 2740.7998  486.6656 2015.2142  608.3465  715.5026

In [20]:
GitHub_RMSE_No_Watchers <- tibble(
    Model = "Full Regression No Watchers",
    RMSE = rmse(model = GitHub_No_Watchers,
                data = testing_GitHub))

GitHub_RMSE_No_Watchers

Model,RMSE
<chr>,<dbl>
Full Regression No Watchers,2759.662


*Since we saw in the previous assignment visualization that 'Watchers' was perfectly linearly correlated with Stars.  Now, let's fit a simple linear regression model with only 'Forks' as an input.*

In [21]:
GitHub_Simple_model <- lm(Stars ~ Forks, data = training_GitHub)

GitHub_Simple_model


Call:
lm(formula = Stars ~ Forks, data = training_GitHub)

Coefficients:
(Intercept)        Forks  
    339.489        3.219  


In [22]:
GitHub_test_pred_Simple <- predict(GitHub_full, newdata = testing_GitHub)

head(GitHub_test_pred_full)

1    2    3    4    5    6 
 418 2838  532 1029  289  206

In [23]:
GitHub_RMSE_Simple_models <- tibble(
    Model = "OLS Simple Regression",
    RMSE = rmse(model = GitHub_Simple_model,
                data = testing_GitHub))

GitHub_RMSE_Simple_models

Model,RMSE
<chr>,<dbl>
OLS Simple Regression,2692.918
